In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import sys

sys.path.append("../../../")
import os
from pathlib import Path

import geopandas as gpd
import pandas as pd

from povertymapping.iso3 import get_region_name
from povertymapping.rollout_grids import get_region_filtered_bingtile_grids

/home/jace/workspace/unicef-ai4d-relative-wealth/env/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


# Generate Rollout Grids: Thailand

This notebook is the second step in the rollout and generates the rollout grid tiles over the country. The output file is used as an input for Step 3, where we will run the trained model over the set of grids.

The generated grids are set at 2.4km (zoom level 14), matching the grids used during model training. The grids are also filtered to only include populated areas based on Meta's High Resolution Settlement Layer (HRSL) data.

## Set country-specific parameters

In [3]:
COUNTRY_CODE = "th"
REGION = get_region_name(COUNTRY_CODE, code="alpha-2").lower()
ADMIN_LVL = "ADM2"
ZOOM_LEVEL = 14

## Generate Grids

In [6]:
admin_grids_gdf = get_region_filtered_bingtile_grids(
    REGION, admin_lvl=ADMIN_LVL, quadkey_lvl=ZOOM_LEVEL, use_cache=True
)

2023-03-09 18:01:04.437 | INFO     | povertymapping.rollout_grids:get_region_filtered_bingtile_grids:264 - No cached grids file found. Generating grids file :/home/jace/.cache/geowrangler/quadkey_grids/thailand_14_ADM2_populated_admin_grids.geojson
2023-03-09 18:01:04.440 | DEBUG    | povertymapping.rollout_grids:get_region_filtered_bingtile_grids:281 - Loading boundaries for region thailand and admin level ADM2
2023-03-09 18:01:08.440 | INFO     | povertymapping.rollout_grids:get_region_filtered_bingtile_grids:287 - Generating grids for region thailand and admin level ADM2 at quadkey level 14


2023-03-09 18:02:20.157 | INFO     | povertymapping.rollout_grids:get_region_filtered_bingtile_grids:293 - Generated 96111 grids for region thailand and admin level ADM2 at quadkey level 14
2023-03-09 18:02:20.176 | INFO     | povertymapping.rollout_grids:get_region_filtered_bingtile_grids:299 - Assigning grids to admin areas using metric crs epsg:3857
2023-03-09 18:02:37.709 | INFO     | povertymapping.rollout_grids:get_region_filtered_bingtile_grids:305 - Getting thailand population data for filtering grids
2023-03-09 18:02:38.985 | INFO     | povertymapping.rollout_grids:get_region_filtered_bingtile_grids:307 - Computing population zonal stats per grid
2023-03-09 18:02:38.986 | INFO     | povertymapping.rollout_grids:compute_raster_stats:98 - Creating raster zonal stats for 96111 grids for file size 14930842124
2023-03-09 18:24:51.892 | INFO     | povertymapping.rollout_grids:get_region_filtered_bingtile_grids:318 - Filtering unpopulated grids based on population data
2023-03-09 18:

## Explore Populated Grids

In [7]:
admin_grids_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 78254 entries, 4 to 96110
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   geometry    78254 non-null  geometry
 1   quadkey     78254 non-null  object  
 2   shapeName   78254 non-null  object  
 3   shapeISO    78254 non-null  object  
 4   shapeID     78254 non-null  object  
 5   shapeGroup  78254 non-null  object  
 6   shapeType   78254 non-null  object  
 7   pop_count   78254 non-null  float64 
dtypes: float64(1), geometry(1), object(6)
memory usage: 5.4+ MB


In [8]:
admin_grids_gdf.head(2)

,geometry,quadkey,shapeName,shapeISO,shapeID,shapeGroup,shapeType,pop_count
4,"POLYGON ((98.34961 7.60211, 98.34961 7.62389, ...",13222121311300,Mueang Phuket,None,THA-ADM2-3_0_0-B565,THA,ADM2,755.127425
6,"POLYGON ((98.37158 7.60211, 98.37158 7.62389, ...",13222121311301,Mueang Phuket,None,THA-ADM2-3_0_0-B565,THA,ADM2,449.253025


In [ ]:
# Uncomment to view the generated grid on an interactive map
# admin_grids_gdf.explore()

## Save to file

In [ ]:
rollout_date = "-".join(os.getcwd().split("/")[-2].split("-")[:3])
grid_save_path = f"./{rollout_date}-{COUNTRY_CODE}-rollout-grids.geojson"
admin_grids_gdf.to_file(grid_save_path, driver="GeoJSON", index=False)